# Python を使ってネットワーク科学の問題を解決しよう
Chapter 3. 通信シミュレーション

----

明治大学理工学部情報科学科  
飯塚 秀明

## 演習前に
この演習を行う前に、下記の注ぎ口を一度実行してください。
演習に必要な各種機能が自動的に導入されます。

In [ ]:
!pip install git+https://github.com/kazh98/ybook.git

## 可視化ツールdraw
**【課題】まずは、下記の注ぎ口を実行してみましょう。**

In [ ]:
%matplotlib inline
import numpy as np
from ybook import draw

draw(np.array([2, 3]), [(0, 1), (1, 2), (0, 2)], np.array([0, 1, 2]))

実行に成功すると、``LINK STATUS`` と``FLOW STATUS`` という２つのデータ、加えて１つの図が出てくると思います。
これらは、ネットワークのある状態を表しています。

`draw` 命令をひとつずつ紐解いていきましょう。
  * 図の青い丸は**ノード**(地点)、ノードを結ぶ黒い線分は**リンク**(通信線)、そしてその上下にある矢印はリンクを流れる**通信**を表します。
  * 第１引数`np.array([2, 3])` は、**リンクの許容容量** を左から順に指定しています。この場合、左からリンク$c_0$ は2、リンク$c_1$ は3 の許容容量をもつことを意味しています。
  * 第２引数`[(0, 1), (1, 2), (0, 2)]` は、各ノードを左から順に番号付けたとしての**通信の端点** を指定しています。最初の通信`(0, 1)` は、通信$x_0$ がノード0(左のノード) からノード1(中央のノード) へ流れることを、２番目の通信`(1, 2)` は、通信$x_1$ がノード1(中央のノード) からノード2(右のノード) へ流れることを、最後の通信`(0, 2)` は、通信$x_2$ がノード0(左のノード) からノード2(右のノード) へ流れることを意味しています。
  * 第３引数`np.array([0, 1, 2])` は、**通信の使用帯域幅** を表しています。この場合、通信$x_0$ は帯域幅0 (通信なし)、通信$x_1$ は帯域幅1、通信$x_0$ は帯域幅2 をそれぞれ使用していることを意味します。
  * 通信の使用帯域幅については**FLOW STATUS**、リンクの許容容量とリンクの使用帯域幅については**LINK STATUS** の出力を見ても確認することができます。

リンクを表す線分は、もし何も通信が流れていないときには灰色、何か通信が流れているときには黒色、流れている通信がリンクの許容容量を超える場合には赤色で表示されます。

**【課題】変数`x` の値を変更して、リンクが灰色、黒色、赤色となる場合をそれぞれ確認してみましょう。**

In [ ]:
%matplotlib inline
import numpy as np
from ybook import draw

c = np.array([2])
x = np.array([0, 0])


draw(c, [(0, 1), (0, 1)], x)

## KKT条件を用いた3変数の問題の解法
前回勉強したKKT 条件を用いて、３変数の問題を解決しましょう。
ここでは、同じく前回取り扱った次のプログラムで構築されるネットワークを考えます。

**【課題】下記の注ぎ口を実行してみましょう。**

In [ ]:
%matplotlib inline
import numpy as np
from ybook import draw

c = np.array([2, 3])
x = np.array([0, 0, 0])


draw(c, [(0, 1), (1, 2), (0, 2)], x)

このネットワークの転送レート最適化を、KKT条件を用いて解くことを考えましょう。

**【課題】下記の注ぎ口を編集し、ラグランジュ関数$L$ とそれから求められるKKT条件に基づく連立方程式を解くプログラムを完成させ、実行してみましょう。**

In [ ]:
%matplotlib inline
import numpy as np
from ybook import draw
from sympy import *
init_printing()

x0, x1, x2, l0, l1 = symbols("x_0 x_1 x_2 l_0 l_1", nonnegative=True)

c = np.array([2, 3])
L = ???
result = solve([
    ???
])[0]
x = np.array([result[x0], result[x1], result[x2]])

draw(c, [(0, 1), (1, 2), (0, 2)], x)

## 最適化ソルバーによる解法
いままでは、KKT条件を用いて、

  * 最適化問題 -> 連立方程式
 
という変換を行い、転送レート最適化問題を解きました。

**最適化ソルバー** を用いると、最適化問題を直接与えて問題を解くことができます。

## 最適化ソルバーの使用例
ここでは、次の最適化問題、
\begin{align}
\text{求む: }
& \log x_0 + \log x_1 + \log x_2 \rightarrow \text{最大}, \\
\text{制約: }
& x_0 + x_2 \le 2, \\
& x_1 + x_2 \le 3, \\
& 0 \le x_0, x_1, x_2.
\end{align}
を例に、最適化ソルバーを用いた解法を確認しましょう。
ソルバーに入力するため、まず次のように最適化問題を変形します。
\begin{align}
\text{求む: }
& -\log x_0 - \log x_1 - \log x_2 \rightarrow \text{最小}, \\
\text{制約: }
& 0 \le 2 - x_0 - x_2, \\
& 0 \le 3 - x_1 - x_2, \\
& 0 \le x_0, x_1, x_2.
\end{align}
ポイントは、

  * 目的関数は、**最小化** することを考える。
  * 制約は、**0以上の形** に統一する。
  
です。
この形に変形できたら、最適化ソルバー``scipy.optimize.minimize`` を使うことができます。
使い方は下記の通りです。
```python
import numpy as np
from scipy.optimize import Bounds, minimize

minimize(
    lambda x: [最小化したい目的関数],
    [初期点],
    bounds=Bounds([最小値], [最大値]),
    constraints=[
        {'type': 'ineq', 'fun': lambda x: [制約式]},
        ...
    ]).x
```
目的関数はそのまま指定すればよく、制約については右辺の式を`[制約式]` に記述します。
変数の取りうる値に上下限がある場合(ここでは非負制約なので下限0ですね)、`[最小値]`、`[最大値]`にこれを指定できます。
初期点は最適化をする上で重要なパラメータですが、ここでは`np.ones([次元数])` を指定すればよいでしょう。

**【課題】先の最適化問題を、最適化ソルバーを用いて解いてみましょう。**

In [ ]:
import numpy as np
from scipy.optimize import Bounds, minimize

minimize(
    lambda x: ???,
    np.ones(3),
    bounds=Bounds(???, np.inf),
    constraints=[
        {'type': 'ineq', 'fun': lambda x: ???},
        {'type': 'ineq', 'fun': lambda x: ???},
    ]).x

## 演習
**【課題】最適化ソルバーと`draw` 関数を用いて、先の問題を解き、結果を表示しましょう。**

**【課題】目的関数の各係数を調整して、より公平な割当を実現してみよう。**

**【課題】リンクの許容容量を変更したときの、割当の変化を調べてみましょう。**

**【課題】ノードが5つあるネットワーク上での帯域幅割当に挑戦してみましょう。**